In [1]:
import pandas as pd
import os, glob

from owlready2 import *
import owlready2
print(owlready2.VERSION)


import importlib.util
import sys
spec = importlib.util.spec_from_file_location("rdfutils", "../../../utils/rdfutils.py")
u = importlib.util.module_from_spec(spec)
sys.modules["rdfutils"] = u
spec.loader.exec_module(u)

from datetime import datetime

def NOW():
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    return "Current Time = "+ str(current_time)

%load_ext autoreload
%autoreload 2


0.40


* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [2]:
def sparql(REQ):
    return list(default_world.sparql(REQ))

In [3]:
onto = get_ontology("WIP_Step1.owl").load()
dIDct = u.createDict(onto) 

In [4]:
comments = u.checkComments(onto)

ID: 0 	 Author: Luc Jonveaux
ID: 1 	 License: CC BY-NC-SA
ID: 2 	 Language: English
ID: 3 	 Project: PROBONO
ID: 4 	 Task: T3.5
ID: 5 	 Description: Creation of a knowledge graph based on a litterature review, augmented by use of LLMs.
ID: 6 	 Changes from: 0.0
ID: 7 	 Changes: Creation of the knowledge graph
ID: 8 	 Changes from: 0.1
ID: 9 	 Changes: Adding relations, simplified ontology, closest items, and descriptions
ID: 10 	 Repository: https://github.com/mm80843/T3.5/
ID: 11 	 TODOs: Quite a number! Creating metaRisks, making it accessible..
ID: 12 	 Creation: 11/11/2023
ID: 13 	 Version: 0.3
ID: 14 	 VersionComment: New blueprints added in 0.3
ID: 15 	 Library: owlready2==0.45


# 1. [x] Doing connections to Risks

### [x] RiskGroup -> TechGroup

In [5]:
with onto:
    class has_RiskGroup_TechGroup(onto.RiskGroup >> onto.TechGroup):
        label = ["Technology groups captured in this risk group"]
        pass
    class has_RiskGroup_TechSubgroup(onto.RiskGroup >> onto.TechSubgroup):
        label = ["Technology subgroups captured in this risk group"]
        pass
    class has_RiskSubgroup_TechGroup(onto.RiskSubgroup >> onto.TechGroup):
        label = ["Technology groups captured in this risk subgroup"]
        pass
    class has_RiskSubgroup_TechSubgroup(onto.RiskSubgroup >> onto.TechSubgroup):
        label = ["Technology subgroups captured in this risk subgroup"]
        pass

In [6]:
for RG in [x.name for x in onto.RiskGroup.instances()]:

    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT ?riskGroup ?techGroup (COUNT(?technology) AS ?techCount)
    WHERE {

    ?riskGroup a pbn:RiskGroup  .
    ?techGroup a pbn:TechGroup  .
    ?technology a pbn:Technology . 
    ?riskGroup a pbn:RiskGroup . 
    ?techSubgroup a pbn:TechSubgroup . 
         
    ?risk pbn:has_RiskGroup ?riskGroup .
    ?risk pbn:has_RiskTechnology ?technology .
    ?technology pbn:has_TechGroup ?techGroup . 
    ?technology pbn:has_TechSubgroup ?techSubgroup . 
            
    FILTER (?riskGroup = pbn:"""+RG+""") 
    }
            
    GROUP BY ?techGroup
    ORDER BY DESC(?techCount)
    LIMIT 3""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_RiskGroup_TechGroup.append(sTG)
        sRG.has_RiskGroup_TechGroup = list(set(sRG.has_RiskGroup_TechGroup))


    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT ?riskGroup ?techSubgroup (COUNT(?technology) AS ?techCount)
    WHERE {

    ?riskGroup a pbn:RiskGroup  .
    ?techGroup a pbn:TechGroup  .
    ?technology a pbn:Technology . 
    ?riskGroup a pbn:RiskGroup . 
    ?techSubgroup a pbn:TechSubgroup . 
            
    ?risk pbn:has_RiskGroup ?riskGroup .
    ?risk pbn:has_RiskTechnology ?technology .
    ?technology pbn:has_TechGroup ?techGroup . 
    ?technology pbn:has_TechSubgroup ?techSubgroup . 
            
    FILTER (?riskGroup = pbn:"""+RG+""") 
    }
            
    GROUP BY ?techSubgroup
    ORDER BY DESC(?techCount)
    LIMIT 10""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_RiskGroup_TechSubgroup.append(sTG)
        sRG.has_RiskGroup_TechSubgroup = list(set(sRG.has_RiskGroup_TechSubgroup))


In [7]:
for RSG in [x.name for x in onto.RiskSubgroup.instances()]:

    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT ?riskSubgroup ?techGroup (COUNT(?technology) AS ?techCount)
    WHERE {

    ?techGroup a pbn:TechGroup  .
    ?technology a pbn:Technology . 
    ?riskSubgroup a pbn:RiskSubgroup . 
         
    ?risk pbn:has_RiskSubgroup ?riskSubgroup .
    ?risk pbn:has_RiskTechnology ?technology .
    ?technology pbn:has_TechGroup ?techGroup . 
    ?technology pbn:has_TechSubgroup ?techSubgroup . 
            
    FILTER (?riskSubgroup = pbn:"""+RSG+""") 
    }
            
    GROUP BY ?techGroup
    ORDER BY DESC(?techCount)
    LIMIT 3""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_RiskSubgroup_TechGroup.append(sTG)
        sRG.has_RiskSubgroup_TechGroup = list(set(sRG.has_RiskSubgroup_TechGroup))

    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT ?riskSubgroup ?techSubgroup (COUNT(?technology) AS ?techCount)
    WHERE {

    ?techSubgroup a pbn:TechSubgroup  .
    ?technology a pbn:Technology . 
    ?riskSubgroup a pbn:RiskSubgroup . 
         
    ?risk pbn:has_RiskSubgroup ?riskSubgroup .
    ?risk pbn:has_RiskTechnology ?technology .
    ?technology pbn:has_TechGroup ?techGroup . 
    ?technology pbn:has_TechSubgroup ?techSubgroup . 
            
    FILTER (?riskSubgroup = pbn:"""+RSG+""") 
    }
            
    GROUP BY ?techSubgroup
    ORDER BY DESC(?techCount)
    LIMIT 10""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_RiskSubgroup_TechSubgroup.append(sTG)
        sRG.has_RiskSubgroup_TechSubgroup = list(set(sRG.has_RiskSubgroup_TechSubgroup))


### [x] RiskGroups -> BP_Interventions

In [8]:
with onto:
    class has_RiskGroup_BPIntervention(onto.RiskGroup >> onto.BP_Intervention):
        label = ["Inteventions captured in this risk group"]
        pass
    class has_RiskSubgroup_BPIntervention(onto.RiskGroup >> onto.BP_Intervention):
        label = ["Inteventions captured in this risk subgroup"]
        pass


In [9]:
for RG in [x.name for x in onto.RiskGroup.instances()]:

    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

        SELECT DISTINCT ?riskGroup ?interv  (COUNT(?mitig) AS ?mCount)
        WHERE {

        ?riskGroup a pbn:RiskGroup  .
        ?risk a pbn:Risk  .
        ?mitig a pbn:Mitigation  .
        ?interv a pbn:BP_Intervention  .
        
        ?risk pbn:has_RiskGroup ?riskGroup  .
        ?risk pbn:has_RiskMitigation ?mitig .
        ?interv pbn:has_InterventionMitig ?mitig  .
        FILTER (?riskGroup = pbn:"""+RG+""") 
        }
        
        GROUP BY ?interv
        ORDER BY DESC(?mCount)           

        LIMIT 5""")

    for a in A:
        [sRG, sTG, c] = a
        sRG.has_RiskGroup_BPIntervention.append(sTG)
        sRG.has_RiskGroup_BPIntervention = list(set(sRG.has_RiskGroup_BPIntervention))


In [10]:
A

[[WIP_Step1.PBN__RiskGroup_8, WIP_Step1.PBN__BP_Intervention_38, 50],
 [WIP_Step1.PBN__RiskGroup_8, WIP_Step1.PBN__BP_Intervention_37, 43],
 [WIP_Step1.PBN__RiskGroup_8, WIP_Step1.PBN__BP_Intervention_22, 16],
 [WIP_Step1.PBN__RiskGroup_8, WIP_Step1.PBN__BP_Intervention_29, 13],
 [WIP_Step1.PBN__RiskGroup_8, WIP_Step1.PBN__BP_Intervention_30, 12]]

In [11]:
for RSG in [x.name for x in onto.RiskSubgroup.instances()]:

    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

        SELECT DISTINCT ?riskSubgroup ?interv  (COUNT(?mitig) AS ?mCount)
        WHERE {

        ?riskSubgroup a pbn:RiskSubgroup  .
        ?risk a pbn:Risk  .
        ?mitig a pbn:Mitigation  .
        ?interv a pbn:BP_Intervention  .
        
        ?risk pbn:has_RiskSubgroup ?riskSubgroup  .
        ?risk pbn:has_RiskMitigation ?mitig .
        ?interv pbn:has_InterventionMitig ?mitig  .
        FILTER (?riskSubgroup = pbn:"""+RSG+""") 
        }
        
        GROUP BY ?interv
        ORDER BY DESC(?mCount)           

        LIMIT 5""")

    for a in A:
        [sRG, sTG, c] = a
        sRG.has_RiskGroup_BPIntervention.append(sTG)
        sRG.has_RiskGroup_BPIntervention = list(set(sRG.has_RiskGroup_BPIntervention))


In [12]:
A

[[WIP_Step1.PBN__RiskSubgroup_85, WIP_Step1.PBN__BP_Intervention_1, 2],
 [WIP_Step1.PBN__RiskSubgroup_85, WIP_Step1.PBN__BP_Intervention_37, 1],
 [WIP_Step1.PBN__RiskSubgroup_85, WIP_Step1.PBN__BP_Intervention_22, 1],
 [WIP_Step1.PBN__RiskSubgroup_85, WIP_Step1.PBN__BP_Intervention_9, 1]]

### [x] RiskGroups -> Owners

In [13]:
with onto:
    class has_RiskGroup_OwnerGroup(onto.RiskGroup >> onto.StakeholderGroup):
        label = ["People groups who can act on this risk group"]
        pass
    class has_RiskGroup_OwnerSubgroup(onto.RiskGroup >> onto.StakeholderSubgroup):
        label = ["People subgroups who can act on this risk group"]
        pass
    class has_RiskSubgroup_OwnerGroup(onto.RiskSubgroup >> onto.StakeholderGroup):
        label = ["People groups who can act on this risk subgroup"]
        pass
    class has_RiskSubgroup_OwnerSubgroup(onto.RiskSubgroup >> onto.StakeholderSubgroup):
        label = ["People subgroups who can act on this risk subgroup"]
        pass

In [14]:
for RG in [x.name for x in onto.RiskGroup.instances()]:

    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT ?riskGroup ?shGroup (COUNT(?sh) AS ?sCount)
    WHERE {

    ?shGroup a pbn:StakeholderGroup  .
    ?sh a pbn:Stakeholder . 
    ?riskGroup a pbn:RiskGroup . 
         
    ?risk pbn:has_RiskGroup ?riskGroup .
    ?risk pbn:has_RiskOwner ?sh .
    ?sh pbn:has_StakeholderGroup ?shGroup . 
    ?sh pbn:has_StakeholderSubgroup ?shSubgroup . 
            
    FILTER (?riskGroup = pbn:"""+RG+""") 
    }
            
    GROUP BY ?shGroup
    ORDER BY DESC(?sCount)
    LIMIT 3""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_RiskGroup_OwnerGroup.append(sTG)
        sRG.has_RiskGroup_OwnerGroup = list(set(sRG.has_RiskGroup_OwnerGroup))


    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT ?riskGroup ?shSubgroup (COUNT(?sh) AS ?sCount)
    WHERE {

    ?shSubgroup a pbn:StakeholderSubgroup  .
    ?sh a pbn:Stakeholder . 
    ?riskGroup a pbn:RiskGroup . 
         
    ?risk pbn:has_RiskGroup ?riskGroup .
    ?risk pbn:has_RiskOwner ?sh .
    ?sh pbn:has_StakeholderGroup ?shGroup . 
    ?sh pbn:has_StakeholderSubgroup ?shSubgroup . 
            
    FILTER (?riskGroup = pbn:"""+RG+""") 
    }
            
    GROUP BY ?shSubgroup
    ORDER BY DESC(?sCount)
    LIMIT 10""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_RiskGroup_OwnerGroup.append(sTG)
        sRG.has_RiskGroup_OwnerGroup = list(set(sRG.has_RiskGroup_OwnerGroup))

In [15]:
for RSG in [x.name for x in onto.RiskSubgroup.instances()]:

    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT ?riskSubgroup ?shGroup (COUNT(?sh) AS ?sCount)
    WHERE {

    ?shGroup a pbn:StakeholderGroup  .
    ?sh a pbn:Stakeholder . 
    ?riskSubgroup a pbn:RiskSubgroup . 
         
    ?risk pbn:has_RiskSubgroup ?riskSubgroup .
    ?risk pbn:has_RiskOwner ?sh .
    ?sh pbn:has_StakeholderGroup ?shGroup . 
    ?sh pbn:has_StakeholderSubgroup ?shSubgroup . 
            
    FILTER (?riskSubgroup = pbn:"""+RSG+""") 
    }
            
    GROUP BY ?shGroup
    ORDER BY DESC(?sCount)
    LIMIT 3""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_RiskSubgroup_OwnerGroup.append(sTG)
        sRG.has_RiskSubgroup_OwnerGroup = list(set(sRG.has_RiskSubgroup_OwnerGroup))


    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT ?riskSubgroup ?shSubgroup (COUNT(?sh) AS ?sCount)
    WHERE {

    ?shSubgroup a pbn:StakeholderSubgroup  .
    ?sh a pbn:Stakeholder . 
    ?riskSubgroup a pbn:RiskSubgroup . 
         
    ?risk pbn:has_RiskSubgroup ?riskSubgroup .
    ?risk pbn:has_RiskOwner ?sh .
    ?sh pbn:has_StakeholderGroup ?shGroup . 
    ?sh pbn:has_StakeholderSubgroup ?shSubgroup . 
            
    FILTER (?riskSubgroup = pbn:"""+RSG+""") 
    }
            
    GROUP BY ?shSubgroup
    ORDER BY DESC(?sCount)
    LIMIT 10""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_RiskSubgroup_OwnerGroup.append(sTG)
        sRG.has_RiskSubgroup_OwnerGroup = list(set(sRG.has_RiskSubgroup_OwnerGroup))

### [x] RiskGroups -> Subject

In [16]:
with onto:
    class has_RiskGroup_SubjectGroup(onto.RiskGroup >> onto.StakeholderGroup):
        label = ["People groups who are subject to this risk group"]
        pass
    class has_RiskGroup_SubjectSubgroup(onto.RiskGroup >> onto.StakeholderSubgroup):
        label = ["People subgroups who are subject to this risk group"]
        pass
    class has_RiskSubgroup_SubjectGroup(onto.RiskSubgroup >> onto.StakeholderGroup):
        label = ["People groups who are subject to this risk subgroup"]
        pass
    class has_RiskSubgroup_SubjectSubgroup(onto.RiskSubgroup >> onto.StakeholderSubgroup):
        label = ["People subgroups who are subject to this risk subgroup"]
        pass




In [17]:
for RG in [x.name for x in onto.RiskGroup.instances()]:

    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT ?riskGroup ?shGroup (COUNT(?sh) AS ?sCount)
    WHERE {

    ?shGroup a pbn:StakeholderGroup  .
    ?sh a pbn:Stakeholder . 
    ?riskGroup a pbn:RiskGroup . 
         
    ?risk pbn:has_RiskGroup ?riskGroup .
    ?risk pbn:has_RiskSubject ?sh .
    ?sh pbn:has_StakeholderGroup ?shGroup . 
    ?sh pbn:has_StakeholderSubgroup ?shSubgroup . 
            
    FILTER (?riskGroup = pbn:"""+RG+""") 
    }
            
    GROUP BY ?shGroup
    ORDER BY DESC(?sCount)
    LIMIT 3""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_RiskGroup_SubjectGroup.append(sTG)
        sRG.has_RiskGroup_SubjectGroup = list(set(sRG.has_RiskGroup_SubjectGroup))


    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT ?riskGroup ?shSubgroup (COUNT(?sh) AS ?sCount)
    WHERE {

    ?shSubgroup a pbn:StakeholderSubgroup  .
    ?sh a pbn:Stakeholder . 
    ?riskGroup a pbn:RiskGroup . 
         
    ?risk pbn:has_RiskGroup ?riskGroup .
    ?risk pbn:has_RiskSubject ?sh .
    ?sh pbn:has_StakeholderGroup ?shGroup . 
    ?sh pbn:has_StakeholderSubgroup ?shSubgroup . 
            
    FILTER (?riskGroup = pbn:"""+RG+""") 
    }
            
    GROUP BY ?shSubgroup
    ORDER BY DESC(?sCount)
    LIMIT 10""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_RiskGroup_SubjectSubgroup.append(sTG)
        sRG.has_RiskGroup_SubjectSubgroup = list(set(sRG.has_RiskGroup_SubjectSubgroup))

In [18]:
for RSG in [x.name for x in onto.RiskSubgroup.instances()]:

    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT ?riskSubgroup ?shGroup (COUNT(?sh) AS ?sCount)
    WHERE {

    ?shGroup a pbn:StakeholderGroup  .
    ?sh a pbn:Stakeholder . 
    ?riskSubgroup a pbn:RiskSubgroup . 
         
    ?risk pbn:has_RiskSubgroup ?riskSubgroup .
    ?risk pbn:has_RiskOwner ?sh .
    ?sh pbn:has_StakeholderGroup ?shGroup . 
    ?sh pbn:has_StakeholderSubgroup ?shSubgroup . 
            
    FILTER (?riskSubgroup = pbn:"""+RSG+""") 
    }
            
    GROUP BY ?shGroup
    ORDER BY DESC(?sCount)
    LIMIT 3""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_RiskSubgroup_SubjectGroup.append(sTG)
        sRG.has_RiskSubgroup_SubjectGroup = list(set(sRG.has_RiskSubgroup_SubjectGroup))


    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT ?riskSubgroup ?shSubgroup (COUNT(?sh) AS ?sCount)
    WHERE {

    ?shSubgroup a pbn:StakeholderSubgroup  .
    ?sh a pbn:Stakeholder . 
    ?riskSubgroup a pbn:RiskSubgroup . 
         
    ?risk pbn:has_RiskSubgroup ?riskSubgroup .
    ?risk pbn:has_RiskOwner ?sh .
    ?sh pbn:has_StakeholderGroup ?shGroup . 
    ?sh pbn:has_StakeholderSubgroup ?shSubgroup . 
            
    FILTER (?riskSubgroup = pbn:"""+RSG+""") 
    }
            
    GROUP BY ?shSubgroup
    ORDER BY DESC(?sCount)
    LIMIT 10""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_RiskSubgroup_SubjectSubgroup.append(sTG)
        sRG.has_RiskSubgroup_SubjectSubgroup = list(set(sRG.has_RiskSubgroup_SubjectSubgroup))

# 2. Connections from Techs

##  [x] Techs -> Stakeholders

In [19]:
with onto:
    class has_TechGroup_StakeholderGroup(onto.TechGroup >> onto.StakeholderGroup):
        label = ["People groups who can use or are impacted by this technology group"]
        pass
    class has_TechGroup_StakeholderSubgroup(onto.TechGroup >> onto.StakeholderSubgroup):
        laebl = ["People subgroups who can use or are impacted by this technology group"]
        pass


In [20]:
for RG in [x.name for x in onto.TechGroup.instances()]:

    print(RG)
    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT   ?techGroup ?shGroup (COUNT(?sh) AS ?sCount)
    WHERE {

    ?shGroup a pbn:StakeholderGroup  .
    ?sh a pbn:Stakeholder . 
    ?risk a pbn:Risk . 
    ?techGroup a pbn:TechGroup .

                    
    ?risk ?o ?sh .
               
    ?sh pbn:has_StakeholderGroup ?shGroup . 
    ?sh pbn:has_StakeholderSubgroup ?shSubgroup . 
    ?risk pbn:has_RiskTechnology ?technology .
    ?technology pbn:has_TechGroup ?techGroup . 

                              
    FILTER (?techGroup = pbn:"""+RG+""") 
    }
            
    GROUP BY ?shGroup
    ORDER BY DESC(?sCount)
    LIMIT 3""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_TechGroup_StakeholderGroup.append(sTG)
        sRG.has_TechGroup_StakeholderGroup = list(set(sRG.has_TechGroup_StakeholderGroup))
    print(A)

    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT  ?techGroup ?shSubgroup (COUNT(?sh) AS ?sCount)
    WHERE {

    ?shSubgroup a pbn:StakeholderSubgroup  .
    ?sh a pbn:Stakeholder . 
    ?risk a pbn:Risk . 
    ?techGroup a pbn:TechGroup .
                               
    ?risk ?o ?sh .
    ?sh pbn:has_StakeholderSubgroup ?shSubgroup . 
    ?risk pbn:has_RiskTechnology ?technology .
    ?technology pbn:has_TechGroup ?techGroup . 
                              
    FILTER (?techGroup = pbn:"""+RG+""") 
    }
            
    GROUP BY ?shSubgroup
    ORDER BY DESC(?sCount)
    LIMIT 10""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_TechGroup_StakeholderSubgroup.append(sTG)
        sRG.has_TechGroup_StakeholderSubgroup = list(set(sRG.has_TechGroup_StakeholderSubgroup))
    print(A)
#17s

PBN__TechGroup_12
[[WIP_Step1.PBN__TechGroup_12, WIP_Step1.PBN__StakeholderGroup_0, 2942], [WIP_Step1.PBN__TechGroup_12, WIP_Step1.PBN__StakeholderGroup_12, 2744], [WIP_Step1.PBN__TechGroup_12, WIP_Step1.PBN__StakeholderGroup_6, 2599]]
[[WIP_Step1.PBN__TechGroup_12, WIP_Step1.PBN__StakeholderSubgroup_0, 1803], [WIP_Step1.PBN__TechGroup_12, WIP_Step1.PBN__StakeholderSubgroup_21, 1288], [WIP_Step1.PBN__TechGroup_12, WIP_Step1.PBN__StakeholderSubgroup_8, 885], [WIP_Step1.PBN__TechGroup_12, WIP_Step1.PBN__StakeholderSubgroup_47, 667], [WIP_Step1.PBN__TechGroup_12, WIP_Step1.PBN__StakeholderSubgroup_95, 640], [WIP_Step1.PBN__TechGroup_12, WIP_Step1.PBN__StakeholderSubgroup_42, 612], [WIP_Step1.PBN__TechGroup_12, WIP_Step1.PBN__StakeholderSubgroup_40, 598], [WIP_Step1.PBN__TechGroup_12, WIP_Step1.PBN__StakeholderSubgroup_49, 488], [WIP_Step1.PBN__TechGroup_12, WIP_Step1.PBN__StakeholderSubgroup_120, 389], [WIP_Step1.PBN__TechGroup_12, WIP_Step1.PBN__StakeholderSubgroup_38, 388]]
PBN__TechGro

In [21]:
with onto:
    class has_TechSubgroup_StakeholderGroup(onto.TechSubgroup >> onto.StakeholderGroup):
        label = ["People groups who can use or are impacted by this technology subgroup"]
        pass
    class has_TechSubgroup_StakeholderSubgroup(onto.TechSubgroup >> onto.StakeholderSubgroup):
        label = ["People subgroups who can use or are impacted by this technology subgroup"]
        pass


In [22]:
for RG in [x.name for x in onto.TechSubgroup.instances()]:

    print(RG)
    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT   ?techSubgroup ?shGroup (COUNT(?sh) AS ?sCount)
    WHERE {

    ?shGroup a pbn:StakeholderGroup  .
    ?sh a pbn:Stakeholder . 
    ?risk a pbn:Risk . 
    ?techSubgroup a pbn:TechSubgroup .
                    
    ?risk ?o ?sh .
    ?sh pbn:has_StakeholderGroup ?shGroup . 
    ?risk pbn:has_RiskTechnology ?technology .
    ?technology pbn:has_TechSubgroup ?techSubgroup . 
                              
    FILTER (?techSubgroup = pbn:"""+RG+""") 
    }
            
    GROUP BY ?shGroup
    ORDER BY DESC(?sCount)
    LIMIT 3""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_TechSubgroup_StakeholderGroup.append(sTG)
        sRG.has_TechSubgroup_StakeholderGroup = list(set(sRG.has_TechSubgroup_StakeholderGroup))
    print(A)

    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT  ?techSubgroup ?shSubgroup (COUNT(?sh) AS ?sCount)
    WHERE {

    ?shSubgroup a pbn:StakeholderSubgroup  .
    ?sh a pbn:Stakeholder . 
    ?risk a pbn:Risk . 
    ?techSubgroup a pbn:TechSubgroup .
                               
    ?risk ?o ?sh .
    ?sh pbn:has_StakeholderSubgroup ?shSubgroup . 
    ?risk pbn:has_RiskTechnology ?technology .
    ?technology pbn:has_TechSubgroup ?techSubgroup . 
                              
    FILTER (?techSubgroup = pbn:"""+RG+""") 
    }
            
    GROUP BY ?shSubgroup
    ORDER BY DESC(?sCount)
    LIMIT 10""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_TechSubgroup_StakeholderSubgroup.append(sTG)
        sRG.has_TechSubgroup_StakeholderSubgroup = list(set(sRG.has_TechSubgroup_StakeholderSubgroup))
    print(A)
#3mins46 initially // 5.42

PBN__TechSubgroup_75
[[WIP_Step1.PBN__TechSubgroup_75, WIP_Step1.PBN__StakeholderGroup_0, 308], [WIP_Step1.PBN__TechSubgroup_75, WIP_Step1.PBN__StakeholderGroup_12, 304], [WIP_Step1.PBN__TechSubgroup_75, WIP_Step1.PBN__StakeholderGroup_6, 238]]
[[WIP_Step1.PBN__TechSubgroup_75, WIP_Step1.PBN__StakeholderSubgroup_21, 194], [WIP_Step1.PBN__TechSubgroup_75, WIP_Step1.PBN__StakeholderSubgroup_0, 153], [WIP_Step1.PBN__TechSubgroup_75, WIP_Step1.PBN__StakeholderSubgroup_95, 76], [WIP_Step1.PBN__TechSubgroup_75, WIP_Step1.PBN__StakeholderSubgroup_8, 71], [WIP_Step1.PBN__TechSubgroup_75, WIP_Step1.PBN__StakeholderSubgroup_47, 60], [WIP_Step1.PBN__TechSubgroup_75, WIP_Step1.PBN__StakeholderSubgroup_38, 56], [WIP_Step1.PBN__TechSubgroup_75, WIP_Step1.PBN__StakeholderSubgroup_49, 53], [WIP_Step1.PBN__TechSubgroup_75, WIP_Step1.PBN__StakeholderSubgroup_34, 52], [WIP_Step1.PBN__TechSubgroup_75, WIP_Step1.PBN__StakeholderSubgroup_42, 52], [WIP_Step1.PBN__TechSubgroup_75, WIP_Step1.PBN__StakeholderSu

##  [x] Techs -> Mitigations

In [23]:
with onto:
    class has_TechGroup_BPIntervention(onto.TechGroup >> onto.BP_Intervention):
        label = ["Interventions enabled by this technology group"]
        pass
    class has_TechSubgroup_BPIntervention(onto.TechSubgroup >> onto.BP_Intervention):
        label = ["Interventions enabled by this technology subgroup"]
        pass


In [24]:
for RG in [x.name for x in onto.TechGroup.instances()]:

    print(RG)
    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT   ?techGroup ?interv (COUNT(?risk) AS ?sCount)
    WHERE {

    ?techGroup a pbn:TechGroup .
    ?risk a pbn:Risk . 
    ?techSubgroup a pbn:TechSubgroup .
               
    ?technology pbn:has_TechGroup ?techGroup . 
    ?risk pbn:has_RiskTechnology ?technology .
    ?risk pbn:has_RiskMitigation ?mitig .
    ?interv pbn:has_InterventionMitig ?mitig  .                

                              
    FILTER (?techGroup = pbn:"""+RG+""") 
    }
            
    GROUP BY ?interv
    ORDER BY DESC(?sCount)
    LIMIT 3""")

    for a in A:
        [sRG, sTG, c] = a
        sRG.has_TechGroup_BPIntervention.append(sTG)
        sRG.has_TechGroup_BPIntervention = list(set(sRG.has_TechGroup_BPIntervention))
    print(A)


PBN__TechGroup_12
[[WIP_Step1.PBN__TechGroup_12, WIP_Step1.PBN__BP_Intervention_29, 69850], [WIP_Step1.PBN__TechGroup_12, WIP_Step1.PBN__BP_Intervention_38, 66929], [WIP_Step1.PBN__TechGroup_12, WIP_Step1.PBN__BP_Intervention_37, 63627]]
PBN__TechGroup_4
[[WIP_Step1.PBN__TechGroup_4, WIP_Step1.PBN__BP_Intervention_1, 57404], [WIP_Step1.PBN__TechGroup_4, WIP_Step1.PBN__BP_Intervention_38, 55372], [WIP_Step1.PBN__TechGroup_4, WIP_Step1.PBN__BP_Intervention_24, 41910]]
PBN__TechGroup_0
[[WIP_Step1.PBN__TechGroup_0, WIP_Step1.PBN__BP_Intervention_38, 161036], [WIP_Step1.PBN__TechGroup_0, WIP_Step1.PBN__BP_Intervention_29, 155575], [WIP_Step1.PBN__TechGroup_0, WIP_Step1.PBN__BP_Intervention_30, 139827]]
PBN__TechGroup_7
[[WIP_Step1.PBN__TechGroup_7, WIP_Step1.PBN__BP_Intervention_38, 149606], [WIP_Step1.PBN__TechGroup_7, WIP_Step1.PBN__BP_Intervention_37, 52070], [WIP_Step1.PBN__TechGroup_7, WIP_Step1.PBN__BP_Intervention_33, 40386]]
PBN__TechGroup_2
[[WIP_Step1.PBN__TechGroup_2, WIP_Step1.

In [25]:
for RG in [x.name for x in onto.TechSubgroup.instances()]:
    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT  ?techSubgroup ?interv (COUNT(?risk) AS ?sCount)
    WHERE {

    ?risk a pbn:Risk . 
    ?techGroup a pbn:TechGroup .
    ?techSubgroup a pbn:TechSubgroup .
    ?interv a pbn:BP_Intervention .

    ?technology pbn:has_TechSubgroup ?techSubgroup . 
    ?risk pbn:has_RiskTechnology ?technology .                        
    ?risk pbn:has_RiskMitigation ?mitig .
    ?interv pbn:has_InterventionMitig ?mitig  .         
                                
                              
    FILTER (?techSubgroup = pbn:"""+RG+""") 
    }
            
    GROUP BY ?interv
    ORDER BY DESC(?sCount)
    LIMIT 5""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_TechSubgroup_BPIntervention.append(sTG)
        sRG.has_TechSubgroup_BPIntervention = list(set(sRG.has_TechSubgroup_BPIntervention))
    print(A)
#3mins46 initially // 5.42

[[WIP_Step1.PBN__TechSubgroup_75, WIP_Step1.PBN__BP_Intervention_29, 1905], [WIP_Step1.PBN__TechSubgroup_75, WIP_Step1.PBN__BP_Intervention_1, 1695], [WIP_Step1.PBN__TechSubgroup_75, WIP_Step1.PBN__BP_Intervention_38, 1440], [WIP_Step1.PBN__TechSubgroup_75, WIP_Step1.PBN__BP_Intervention_30, 1440], [WIP_Step1.PBN__TechSubgroup_75, WIP_Step1.PBN__BP_Intervention_37, 945]]
[[WIP_Step1.PBN__TechSubgroup_42, WIP_Step1.PBN__BP_Intervention_29, 1260], [WIP_Step1.PBN__TechSubgroup_42, WIP_Step1.PBN__BP_Intervention_1, 1230], [WIP_Step1.PBN__TechSubgroup_42, WIP_Step1.PBN__BP_Intervention_38, 735], [WIP_Step1.PBN__TechSubgroup_42, WIP_Step1.PBN__BP_Intervention_17, 735], [WIP_Step1.PBN__TechSubgroup_42, WIP_Step1.PBN__BP_Intervention_42, 585]]
[[WIP_Step1.PBN__TechSubgroup_53, WIP_Step1.PBN__BP_Intervention_1, 1500], [WIP_Step1.PBN__TechSubgroup_53, WIP_Step1.PBN__BP_Intervention_4, 1290], [WIP_Step1.PBN__TechSubgroup_53, WIP_Step1.PBN__BP_Intervention_5, 1215], [WIP_Step1.PBN__TechSubgroup_53

##  [x] Techs -> Risks

In [26]:
with onto:
    class has_TechGroup_RiskGroup(onto.TechGroup >> onto.RiskGroup):
        label = ["Risk groups that can be mitigated with this technology group"]
        pass
    class has_TechGroup_RiskSubgroup(onto.TechGroup >> onto.RiskSubgroup):
        label = ["Risk subgroups that can be mitigated with this technology group"]
        pass
    class has_TechSubgroup_RiskGroup(onto.TechSubgroup >> onto.RiskGroup):
        label = ["Risk groups that can be mitigated with this technology subgroup"]
        pass
    class has_TechSubgroup_RiskSubgroup(onto.TechSubgroup >> onto.RiskSubgroup):
        label = ["Risk subgroups that can be mitigated with this technology subgroup"]
        pass


### TechGroup -> Risks

In [27]:
for RG in [x.name for x in onto.TechGroup.instances()]:

    print(RG)
    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT   ?techGroup ?riskGroup (COUNT(?risk) AS ?sCount)
    WHERE {


    ?riskGroup a pbn:RiskGroup . 
    ?risk a pbn:Risk . 
    ?techGroup a pbn:TechGroup .
                    
    ?risk pbn:has_RiskGroup ?riskGroup .
    ?risk pbn:has_RiskTechnology ?technology .
    ?technology pbn:has_TechGroup ?techGroup . 
                              
    FILTER (?techGroup = pbn:"""+RG+""") 
    }
            
    GROUP BY ?riskGroup
    ORDER BY DESC(?sCount)
    LIMIT 3""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_TechGroup_RiskGroup.append(sTG)
        sRG.has_TechGroup_RiskGroup = list(set(sRG.has_TechGroup_RiskGroup))


    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT  ?techGroup ?riskSubgroup (COUNT(?risk) AS ?sCount)
    WHERE {


    ?riskSubgroup a pbn:RiskSubgroup . 
    ?risk a pbn:Risk . 
    ?techGroup a pbn:TechGroup .
                               
    ?risk pbn:has_RiskSubgroup ?riskSubgroup .
    ?risk pbn:has_RiskTechnology ?technology .
    ?technology pbn:has_TechGroup ?techGroup . 
                              
    FILTER (?techGroup = pbn:"""+RG+""") 
    }
            
    GROUP BY ?riskSubgroup
    ORDER BY DESC(?sCount)
    LIMIT 10""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_TechGroup_RiskSubgroup.append(sTG)
        sRG.has_TechGroup_RiskSubgroup = list(set(sRG.has_TechGroup_RiskSubgroup))
#3mins46 initially // 5.42

PBN__TechGroup_12
PBN__TechGroup_4
PBN__TechGroup_0
PBN__TechGroup_7
PBN__TechGroup_2
PBN__TechGroup_10
PBN__TechGroup_11
PBN__TechGroup_1
PBN__TechGroup_9
PBN__TechGroup_5
PBN__TechGroup_8
PBN__TechGroup_3
PBN__TechGroup_13
PBN__TechGroup_14
PBN__TechGroup_6


### TechSubgroup -> Risks

In [28]:
for RG in [x.name for x in onto.TechSubgroup.instances()]:

    print(RG)
    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT   ?techSubgroup ?riskGroup (COUNT(?risk) AS ?sCount)
    WHERE {


    ?riskGroup a pbn:RiskGroup . 
    ?risk a pbn:Risk . 
    ?techSubgroup a pbn:TechSubgroup .
                    
    ?risk pbn:has_RiskGroup ?riskGroup .
    ?risk pbn:has_RiskTechnology ?technology .
    ?technology pbn:has_TechSubgroup ?techSubgroup . 
                              
    FILTER (?techSubgroup = pbn:"""+RG+""") 
    }
            
    GROUP BY ?riskGroup
    ORDER BY DESC(?sCount)
    LIMIT 3""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_TechSubgroup_RiskGroup.append(sTG)
        sRG.has_TechSubgroup_RiskGroup = list(set(sRG.has_TechSubgroup_RiskGroup))


    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT  ?techSubgroup ?riskSubgroup (COUNT(?risk) AS ?sCount)
    WHERE {


    ?riskSubgroup a pbn:RiskSubgroup . 
    ?risk a pbn:Risk . 
    ?techSubgroup a pbn:TechSubgroup .
                               
    ?risk pbn:has_RiskSubgroup ?riskSubgroup .
    ?risk pbn:has_RiskTechnology ?technology .
    ?technology pbn:has_TechSubgroup ?techSubgroup . 
                              
    FILTER (?techSubgroup = pbn:"""+RG+""") 
    }
            
    GROUP BY ?riskSubgroup
    ORDER BY DESC(?sCount)
    LIMIT 10""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_TechSubgroup_RiskSubgroup.append(sTG)
        sRG.has_TechSubgroup_RiskSubgroup = list(set(sRG.has_TechSubgroup_RiskSubgroup))
#3mins46 initially // 5.42

PBN__TechSubgroup_75
PBN__TechSubgroup_42
PBN__TechSubgroup_53
PBN__TechSubgroup_8
PBN__TechSubgroup_33
PBN__TechSubgroup_67
PBN__TechSubgroup_80
PBN__TechSubgroup_2
PBN__TechSubgroup_10
PBN__TechSubgroup_85
PBN__TechSubgroup_81
PBN__TechSubgroup_50
PBN__TechSubgroup_91
PBN__TechSubgroup_9
PBN__TechSubgroup_1
PBN__TechSubgroup_18
PBN__TechSubgroup_65
PBN__TechSubgroup_113
PBN__TechSubgroup_72
PBN__TechSubgroup_56
PBN__TechSubgroup_102
PBN__TechSubgroup_30
PBN__TechSubgroup_68
PBN__TechSubgroup_74
PBN__TechSubgroup_62
PBN__TechSubgroup_11
PBN__TechSubgroup_77
PBN__TechSubgroup_55
PBN__TechSubgroup_39
PBN__TechSubgroup_3
PBN__TechSubgroup_36
PBN__TechSubgroup_14
PBN__TechSubgroup_16
PBN__TechSubgroup_101
PBN__TechSubgroup_47
PBN__TechSubgroup_52
PBN__TechSubgroup_111
PBN__TechSubgroup_32
PBN__TechSubgroup_51
PBN__TechSubgroup_27
PBN__TechSubgroup_86
PBN__TechSubgroup_59
PBN__TechSubgroup_93
PBN__TechSubgroup_43
PBN__TechSubgroup_76
PBN__TechSubgroup_82
PBN__TechSubgroup_28
PBN__TechSubgr

# 3. Stakeholders

## [x] Stakeholders --> Risks

In [29]:
with onto:
    class has_StakeholderGroup_TechGroup(onto.StakeholderGroup >> onto.TechGroup):
        label = ["Technology groups that can be used this people group"]
        pass
    class has_StakeholderGroup_TechSubgroup(onto.StakeholderGroup >> onto.TechSubgroup):
        label = ["Technology subgroups that can be used this people group"]
        pass
    class has_StakeholderSubgroup_TechGroup(onto.StakeholderSubgroup >> onto.TechGroup):
        label = ["Technology groups that can be used this people subgroup"]
        pass
    class has_StakeholderSubgroup_TechSubgroup(onto.StakeholderSubgroup >> onto.TechSubgroup):
        label = ["Technology subgroups that can be used this people subgroup"]
        pass


In [30]:
with onto:
    class has_StakeholderGroup_RiskGroup_Owner(onto.StakeholderGroup >> onto.RiskGroup):
        label = ["Risk groups that can be mitigated by this people subgroup"]
        pass
    class has_StakeholderGroup_RiskSubgroup_Owner(onto.StakeholderSubgroup >> onto.RiskGroup):
        label = ["Risk subgroups that can be mitigated by this people subgroup"]
        pass
    class has_StakeholderGroup_RiskGroup_Subject(onto.StakeholderGroup >> onto.RiskGroup):
        label = ["Risk subgroups that can be mitigated by this people subgroup"]
        pass
    class has_StakeholderGroup_RiskSubgroup_Subject(onto.StakeholderSubgroup >> onto.RiskGroup):
        label = ["Risk subgroups that can be mitigated by this people subgroup"]
        pass


## [o] Stakeholder -> Techs

In [31]:
if 0:
    for RG in [x.name for x in onto.StakeholderGroup.instances()]:

        print(RG)
        A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

        SELECT  ?shGroup ?techGroup (COUNT(?risk) AS ?sCount)
        WHERE {

        ?shGroup a pbn:StakeholderGroup  .
        ?sh a pbn:Stakeholder . 
        ?riskGroup a pbn:RiskGroup . 
        ?risk a pbn:Risk . 
        ?techGroup a pbn:TechGroup .
        ?techSubgroup a pbn:TechSubgroup .
                        
        ?risk pbn:has_RiskGroup ?riskGroup .
        ?risk pbn:has_RiskOwner ?sh .
        ?sh pbn:has_StakeholderGroup ?shGroup . 
        ?sh pbn:has_StakeholderSubgroup ?shSubgroup . 
        ?risk pbn:has_RiskTechnology ?technology .
        ?technology pbn:has_TechGroup ?techGroup . 
        ?technology pbn:has_TechSubgroup ?techSubgroup . 
                                
        FILTER (?shGroup = pbn:"""+RG+""") 
        }
                
        GROUP BY ?techGroup
        ORDER BY DESC(?sCount)
        LIMIT 3""")
        for a in A:
            [sRG, sTG, c] = a
            sRG.has_StakeholderGroup_TechGroup.append(sTG)
            sRG.has_StakeholderGroup_TechGroup = list(set(sRG.has_StakeholderGroup_TechGroup))


        A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

        SELECT  ?shGroup ?techSubgroup (COUNT(?risk) AS ?sCount)
        WHERE {

        ?shGroup a pbn:StakeholderGroup  .
        ?sh a pbn:Stakeholder . 
        ?riskGroup a pbn:RiskGroup . 
        ?risk a pbn:Risk . 
        ?techGroup a pbn:TechGroup .
        ?techSubgroup a pbn:TechSubgroup .
                                
        ?risk pbn:has_RiskGroup ?riskGroup .
        ?risk pbn:has_RiskOwner ?sh .
        ?sh pbn:has_StakeholderGroup ?shGroup . 
        ?sh pbn:has_StakeholderSubgroup ?shSubgroup . 
        ?risk pbn:has_RiskTechnology ?technology .
        ?technology pbn:has_TechGroup ?techGroup . 
        ?technology pbn:has_TechSubgroup ?techSubgroup . 
                                
        FILTER (?shGroup = pbn:"""+RG+""") 
        }
                
        GROUP BY ?techSubgroup
        ORDER BY DESC(?sCount)
        LIMIT 10""")
        for a in A:
            [sRG, sTG, c] = a
            sRG.has_StakeholderGroup_TechSubgroup.append(sTG)
            sRG.has_StakeholderGroup_TechSubgroup = list(set(sRG.has_StakeholderGroup_TechSubgroup))
    #3mins46 initially // 5.42

### Stakeholder groups

In [32]:
for RG in [x.name for x in onto.StakeholderGroup.instances()]:

    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT  ?shGroup ?riskGroup (COUNT(?risk) AS ?sCount)
    WHERE {

    ?shGroup a pbn:StakeholderGroup  .
    ?sh a pbn:Stakeholder . 
    ?riskGroup a pbn:RiskGroup . 
         
    ?risk pbn:has_RiskGroup ?riskGroup .
    ?risk pbn:has_RiskOwner ?sh .
    ?sh pbn:has_StakeholderGroup ?shGroup . 
    ?sh pbn:has_StakeholderSubgroup ?shSubgroup . 
            
    FILTER (?StakeholderGroup = pbn:"""+RG+""") 
    }
            
    GROUP BY ?riskGroup
    ORDER BY DESC(?sCount)
    LIMIT 3""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_StakeholderGroup_RiskGroup_Owner.append(sTG)
        sRG.has_StakeholderGroup_RiskGroup_Owner = list(set(sRG.has_StakeholderGroup_RiskGroup_Owner))


    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT ?shSubgroup ?riskGroup (COUNT(?risk) AS ?sCount)
    WHERE {

    ?shSubgroup a pbn:StakeholderSubgroup  .
    ?sh a pbn:Stakeholder . 
    ?riskGroup a pbn:RiskGroup . 
         
    ?risk pbn:has_RiskGroup ?riskGroup .
    ?risk pbn:has_RiskOwner ?sh .
    ?sh pbn:has_StakeholderGroup ?shGroup . 
    ?sh pbn:has_StakeholderSubgroup ?shSubgroup . 
            
    FILTER (?StakeholderGroup = pbn:"""+RG+""") 
    }
            
    GROUP BY ?riskGroup
    ORDER BY DESC(?sCount)
    LIMIT 10""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_StakeholderGroup_RiskSubgroup_Owner.append(sTG)
        sRG.has_StakeholderGroup_RiskSubgroup_Owner = list(set(sRG.has_StakeholderGroup_RiskSubgroup_Owner))
    
    ## Subject

    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT  ?shGroup ?riskGroup (COUNT(?risk) AS ?sCount)
    WHERE {

    ?shGroup a pbn:StakeholderGroup  .
    ?sh a pbn:Stakeholder . 
    ?riskGroup a pbn:RiskGroup . 
         
    ?risk pbn:has_RiskGroup ?riskGroup .
    ?risk pbn:has_RiskSubject ?sh .
    ?sh pbn:has_StakeholderGroup ?shGroup . 
    ?sh pbn:has_StakeholderSubgroup ?shSubgroup . 
            
    FILTER (?StakeholderGroup = pbn:"""+RG+""") 
    }
            
    GROUP BY ?riskGroup
    ORDER BY DESC(?sCount)
    LIMIT 3""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_StakeholderGroup_RiskGroup_Subject.append(sTG)
        sRG.has_StakeholderGroup_RiskGroup_Subject = list(set(sRG.has_StakeholderGroup_RiskGroup_Subject))


    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT ?shSubgroup ?riskGroup (COUNT(?risk) AS ?sCount)
    WHERE {

    ?shSubgroup a pbn:StakeholderSubgroup  .
    ?sh a pbn:Stakeholder . 
    ?riskGroup a pbn:RiskGroup . 
         
    ?risk pbn:has_RiskGroup ?riskGroup .
    ?risk pbn:has_RiskSubject ?sh .
    ?sh pbn:has_StakeholderGroup ?shGroup . 
    ?sh pbn:has_StakeholderSubgroup ?shSubgroup . 
            
    FILTER (?StakeholderGroup = pbn:"""+RG+""") 
    }
            
    GROUP BY ?riskGroup
    ORDER BY DESC(?sCount)
    LIMIT 10""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_StakeholderGroup_RiskSubgroup_Subject.append(sTG)
        sRG.has_StakeholderGroup_RiskSubgroup_Subject = list(set(sRG.has_StakeholderGroup_RiskSubgroup_Subject))

### Stakeholder subgroups

In [33]:
with onto:
    class has_StakeholderSubgroup_RiskGroup_Owner(onto.StakeholderGroup >> onto.RiskGroup):
        label = ["Risk groups that can be mitigated by this people subgroup"]
        pass
    class has_StakeholderSubgroup_RiskSubgroup_Owner(onto.StakeholderSubgroup>> onto.RiskGroup ):
        label = ["Risk subgroups that can be mitigated by this people subgroup"]
        pass
    class has_StakeholderSubgroup_RiskGroup_Subject(onto.StakeholderGroup >> onto.RiskGroup):
        label = ["Risk groups that can be impacted by this people subgroup"]
        pass
    class has_StakeholderSubgroup_RiskSubgroup_Subject(onto.StakeholderSubgroup >> onto.RiskGroup):
        label = ["Risk subgroups that can be impacted by this people subgroup"]
        pass

In [34]:
for RG in [x.name for x in onto.StakeholderSubgroup.instances()]:

    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT  ?shSubgroup ?riskGroup (COUNT(?risk) AS ?sCount)
    WHERE {

    ?shGroup a pbn:StakeholderGroup  .
    ?sh a pbn:Stakeholder . 
    ?riskGroup a pbn:RiskGroup . 
         
    ?risk pbn:has_RiskGroup ?riskGroup .
    ?risk pbn:has_RiskOwner ?sh .
    ?sh pbn:has_StakeholderGroup ?shGroup . 
    ?sh pbn:has_StakeholderSubgroup ?shSubgroup . 
            
    FILTER (?shSubgroup = pbn:"""+RG+""") 
    }
            
    GROUP BY ?riskGroup
    ORDER BY DESC(?sCount)
    LIMIT 3""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_StakeholderSubgroup_RiskGroup_Owner.append(sTG)
        sRG.has_StakeholderSubgroup_RiskGroup_Owner = list(set(sRG.has_StakeholderSubgroup_RiskGroup_Owner))


    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT ?shSubgroup ?riskSubgroup (COUNT(?risk) AS ?sCount)
    WHERE {

    ?shSubgroup a pbn:StakeholderSubgroup  .
    ?sh a pbn:Stakeholder . 
    ?riskSubgroup a pbn:RiskSubgroup . 
         
    ?risk pbn:has_RiskSubgroup ?riskSubgroup .
    ?risk pbn:has_RiskOwner ?sh .
    ?sh pbn:has_StakeholderGroup ?shGroup . 
    ?sh pbn:has_StakeholderSubgroup ?shSubgroup . 
            
    FILTER (?shSubgroup = pbn:"""+RG+""") 
    }
            
    GROUP BY ?riskSubgroup
    ORDER BY DESC(?sCount)
    LIMIT 10""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_StakeholderSubgroup_RiskSubgroup_Owner.append(sTG)
        sRG.has_StakeholderSubgroup_RiskSubgroup_Owner = list(set(sRG.has_StakeholderSubgroup_RiskSubgroup_Owner))
    print(A)

[[WIP_Step1.PBN__StakeholderSubgroup_21, WIP_Step1.PBN__RiskSubgroup_2, 284], [WIP_Step1.PBN__StakeholderSubgroup_21, WIP_Step1.PBN__RiskSubgroup_35, 180], [WIP_Step1.PBN__StakeholderSubgroup_21, WIP_Step1.PBN__RiskSubgroup_8, 173], [WIP_Step1.PBN__StakeholderSubgroup_21, WIP_Step1.PBN__RiskSubgroup_7, 160], [WIP_Step1.PBN__StakeholderSubgroup_21, WIP_Step1.PBN__RiskSubgroup_58, 123], [WIP_Step1.PBN__StakeholderSubgroup_21, WIP_Step1.PBN__RiskSubgroup_37, 119], [WIP_Step1.PBN__StakeholderSubgroup_21, WIP_Step1.PBN__RiskSubgroup_23, 99], [WIP_Step1.PBN__StakeholderSubgroup_21, WIP_Step1.PBN__RiskSubgroup_57, 90], [WIP_Step1.PBN__StakeholderSubgroup_21, WIP_Step1.PBN__RiskSubgroup_4, 82], [WIP_Step1.PBN__StakeholderSubgroup_21, WIP_Step1.PBN__RiskSubgroup_63, 66]]
[[WIP_Step1.PBN__StakeholderSubgroup_48, WIP_Step1.PBN__RiskSubgroup_7, 45], [WIP_Step1.PBN__StakeholderSubgroup_48, WIP_Step1.PBN__RiskSubgroup_61, 41], [WIP_Step1.PBN__StakeholderSubgroup_48, WIP_Step1.PBN__RiskSubgroup_64, 4

## [x] Stakeholders --> Mitigations

In [35]:
with onto:
    class has_StakeholderGroup_BPIntervention(onto.StakeholderGroup >> onto.BP_Intervention):
        label = ["Interventions that can be lead by this people group"]
        pass
    class has_StakeholderSubgroup_BPIntervention(onto.StakeholderSubgroup >> onto.BP_Intervention):
        label = ["Interventions that can be lead by this people subgroup"]
        pass


In [36]:
for RG in [x.name for x in onto.StakeholderGroup.instances()]:

    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

        SELECT DISTINCT ?shGroup ?interv  (COUNT(?risk) AS ?mCount)
        WHERE {

        ?shGroup a pbn:StakeholderGroup  .
        ?risk a pbn:Risk  .
        ?mitig a pbn:Mitigation  .
        ?interv a pbn:BP_Intervention  .
               
        ?sh  pbn:has_StakeholderGroup ?shGroup .        
        ?risk pbn:has_RiskOwner ?sh .
        ?risk pbn:has_RiskMitigation ?mitig .
        ?interv pbn:has_InterventionMitig ?mitig  .

        FILTER (?shGroup = pbn:"""+RG+""") 
        }
        
        GROUP BY ?interv
        ORDER BY DESC(?mCount)           

        LIMIT 5""")

    for a in A:
        [sRG, sTG, c] = a
        sRG.has_StakeholderGroup_BPIntervention.append(sTG)
        sRG.has_StakeholderGroup_BPIntervention = list(set(sRG.has_StakeholderGroup_BPIntervention))


In [37]:
for RG in [x.name for x in onto.StakeholderSubgroup.instances()]:

    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

        SELECT DISTINCT ?shSubgroup ?interv  (COUNT(?risk) AS ?mCount)
        WHERE {

        ?shSubgroup a pbn:StakeholderSubgroup  .
        ?risk a pbn:Risk  .
        ?mitig a pbn:Mitigation  .
        ?interv a pbn:BP_Intervention  .
        ?sh a pbn:Stakeholder  .
                       
        ?risk pbn:has_RiskMitigation ?mitig .
        ?risk pbn:has_RiskOwner ?sh .
        ?interv pbn:has_InterventionMitig ?mitig  .
        ?sh  pbn:has_StakeholderSubgroup ?shSubgroup .
               
        FILTER (?shSubgroup = pbn:"""+RG+""") 
        }
        
        GROUP BY ?interv
        ORDER BY DESC(?mCount)           

        LIMIT 5""")

    for a in A:
        [sRG, sTG, c] = a
        sRG.has_StakeholderSubgroup_BPIntervention.append(sTG)
        sRG.has_StakeholderSubgroup_BPIntervention = list(set(sRG.has_StakeholderSubgroup_BPIntervention))


## [x] Stakeholders --> Techs

In [38]:
for RG in [x.name for x in onto.StakeholderGroup.instances()]:

    print(RG)
    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT  ?shGroup ?techGroup (COUNT(?risk) AS ?sCount)
    WHERE {

    ?shGroup a pbn:StakeholderGroup  .
    ?sh a pbn:Stakeholder . 
    ?riskGroup a pbn:RiskGroup . 
    ?risk a pbn:Risk . 
    ?techGroup a pbn:TechGroup .
    ?techSubgroup a pbn:TechSubgroup .
                    
    ?risk pbn:has_RiskGroup ?riskGroup .
    ?risk pbn:has_RiskOwner ?sh .
    ?sh pbn:has_StakeholderGroup ?shGroup . 
    ?sh pbn:has_StakeholderSubgroup ?shSubgroup . 
    ?risk pbn:has_RiskTechnology ?technology .
    ?technology pbn:has_TechGroup ?techGroup . 
    ?technology pbn:has_TechSubgroup ?techSubgroup . 
                              
    FILTER (?shGroup = pbn:"""+RG+""") 
    }
            
    GROUP BY ?techGroup
    ORDER BY DESC(?sCount)
    LIMIT 3""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_StakeholderGroup_TechGroup.append(sTG)
        sRG.has_StakeholderGroup_TechGroup = list(set(sRG.has_StakeholderGroup_TechGroup))


    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT  ?shGroup ?techSubgroup (COUNT(?risk) AS ?sCount)
    WHERE {

    ?shGroup a pbn:StakeholderGroup  .
    ?sh a pbn:Stakeholder . 
    ?riskGroup a pbn:RiskGroup . 
    ?risk a pbn:Risk . 
    ?techGroup a pbn:TechGroup .
    ?techSubgroup a pbn:TechSubgroup .
                               
    ?risk pbn:has_RiskGroup ?riskGroup .
    ?risk pbn:has_RiskOwner ?sh .
    ?sh pbn:has_StakeholderGroup ?shGroup . 
    ?sh pbn:has_StakeholderSubgroup ?shSubgroup . 
    ?risk pbn:has_RiskTechnology ?technology .
    ?technology pbn:has_TechGroup ?techGroup . 
    ?technology pbn:has_TechSubgroup ?techSubgroup . 
                              
    FILTER (?shGroup = pbn:"""+RG+""") 
    }
            
    GROUP BY ?techSubgroup
    ORDER BY DESC(?sCount)
    LIMIT 10""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_StakeholderGroup_TechSubgroup.append(sTG)
        sRG.has_StakeholderGroup_TechSubgroup = list(set(sRG.has_StakeholderGroup_TechSubgroup))
#3mins46 initially // 5.42

PBN__StakeholderGroup_12
PBN__StakeholderGroup_6
PBN__StakeholderGroup_13
PBN__StakeholderGroup_0
PBN__StakeholderGroup_1
PBN__StakeholderGroup_8
PBN__StakeholderGroup_2
PBN__StakeholderGroup_14
PBN__StakeholderGroup_10
PBN__StakeholderGroup_11
PBN__StakeholderGroup_4
PBN__StakeholderGroup_17
PBN__StakeholderGroup_15
PBN__StakeholderGroup_9
PBN__StakeholderGroup_5
PBN__StakeholderGroup_3
PBN__StakeholderGroup_7
PBN__StakeholderGroup_16


In [39]:
for RG in [x.name for x in onto.StakeholderSubgroup.instances()]:
    # @TODO: ceci prend beaucoup trop de temps à traiter
    # @TODO find how to replace/update it
    print(RG)
    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT  ?shSubgroup ?techGroup (COUNT(?risk) AS ?sCount)
    WHERE {

    ?shSubgroup a pbn:StakeholderSubgroup  .
    ?sh a pbn:Stakeholder . 
    ?risk a pbn:Risk . 
    ?techGroup a pbn:TechGroup .
                               
    ?risk ?o ?sh .
    ?sh pbn:has_StakeholderSubgroup ?shSubgroup . 
    ?risk pbn:has_RiskTechnology ?technology .
    ?technology pbn:has_TechGroup ?techGroup . 
            
    FILTER (?shSubgroup = pbn:"""+RG+""") 
    }
            
    GROUP BY ?techGroup
    ORDER BY DESC(?sCount)
    LIMIT 3""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_StakeholderSubgroup_TechGroup.append(sTG)
        sRG.has_StakeholderSubgroup_TechGroup = list(set(sRG.has_StakeholderSubgroup_TechGroup))


    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT ?shSubgroup ?techSubgroup (COUNT(?risk) AS ?sCount)
    WHERE {

    ?shSubgroup a pbn:StakeholderSubgroup  .
    ?sh a pbn:Stakeholder . 
    ?risk a pbn:Risk . 
    ?techSubgroup a pbn:TechSubgroup .                   


    ?sh pbn:has_StakeholderSubgroup ?shSubgroup . 
    ?risk pbn:has_RiskTechnology ?technology .
    ?technology pbn:has_TechSubgroup ?techSubgroup . 
    ?risk ?o ?sh .
                           
    FILTER (?shSubgroup = pbn:"""+RG+""") 
    }
            
    GROUP BY ?techSubgroup
    ORDER BY DESC(?sCount)
    LIMIT 10""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_StakeholderSubgroup_TechSubgroup.append(sTG)
        sRG.has_StakeholderSubgroup_TechSubgroup = list(set(sRG.has_StakeholderSubgroup_TechSubgroup))

PBN__StakeholderSubgroup_21
PBN__StakeholderSubgroup_48
PBN__StakeholderSubgroup_42
PBN__StakeholderSubgroup_95
PBN__StakeholderSubgroup_40
PBN__StakeholderSubgroup_0
PBN__StakeholderSubgroup_38
PBN__StakeholderSubgroup_10
PBN__StakeholderSubgroup_69
PBN__StakeholderSubgroup_44
PBN__StakeholderSubgroup_8
PBN__StakeholderSubgroup_78
PBN__StakeholderSubgroup_24
PBN__StakeholderSubgroup_96
PBN__StakeholderSubgroup_120
PBN__StakeholderSubgroup_119
PBN__StakeholderSubgroup_108
PBN__StakeholderSubgroup_60
PBN__StakeholderSubgroup_31
PBN__StakeholderSubgroup_35
PBN__StakeholderSubgroup_14
PBN__StakeholderSubgroup_50
PBN__StakeholderSubgroup_99
PBN__StakeholderSubgroup_70
PBN__StakeholderSubgroup_1
PBN__StakeholderSubgroup_77
PBN__StakeholderSubgroup_127
PBN__StakeholderSubgroup_7
PBN__StakeholderSubgroup_53
PBN__StakeholderSubgroup_26
PBN__StakeholderSubgroup_54
PBN__StakeholderSubgroup_97
PBN__StakeholderSubgroup_82
PBN__StakeholderSubgroup_74
PBN__StakeholderSubgroup_33
PBN__StakeholderSubg

# [x] 4. Mitigations

## [x] Mitigations --> Risks

In [40]:
with onto:
    class has_BPIntervention_RiskGroup(onto.BP_Intervention >> onto.RiskGroup):
        label = ["Risk groups that can be mitigated by this intervention"]
        pass
    class has_BPIntervention_RiskSubgroup(onto.BP_Intervention >> onto.RiskSubgroup):
        label = ["Risk subgroups that can be mitigated by this intervention"]
        pass


[autoreload of rdfutils failed: Traceback (most recent call last):
  File "/home/kelu/projets/T3.5/venv/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 273, in check
    superreload(m, reload, self.old_objects)
  File "/home/kelu/projets/T3.5/venv/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 471, in superreload
    module = reload(module)
  File "/usr/lib/python3.10/importlib/__init__.py", line 168, in reload
    raise ModuleNotFoundError(f"spec not found for the module {name!r}", name=name)
ModuleNotFoundError: spec not found for the module 'rdfutils'
]


In [41]:
for RG in [x.name for x in onto.BP_Intervention.instances()]:

    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT ?interv ?riskGroup (COUNT(?risk) AS ?techCount)
    WHERE {

    ?riskGroup a pbn:RiskGroup  .
    ?interv a pbn:BP_Intervention  .
    ?risk a pbn:Risk .
               

    ?interv pbn:has_InterventionMitig ?mitig  .    
    ?risk pbn:has_RiskMitigation ?mitig .                    
    ?risk pbn:has_RiskGroup ?riskGroup .

            
    FILTER (?interv = pbn:"""+RG+""") 
    }
            
    GROUP BY ?riskGroup
    ORDER BY DESC(?techCount)
    LIMIT 3""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_BPIntervention_RiskGroup.append(sTG)
        sRG.has_BPIntervention_RiskGroup = list(set(sRG.has_BPIntervention_RiskGroup))


    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT ?interv ?riskSubgroup (COUNT(?risk) AS ?techCount)
    WHERE {

    ?riskSubgroup a pbn:RiskSubgroup  .
    ?interv a pbn:BP_Intervention  .
    ?risk a pbn:Risk .
                         
    ?risk pbn:has_RiskMitigation ?mitig .
    ?interv pbn:has_InterventionMitig ?mitig  .                        
    ?risk pbn:has_RiskSubgroup ?riskSubgroup .
            
    FILTER (?interv = pbn:"""+RG+""") 
    }
            
    GROUP BY ?riskSubgroup
    ORDER BY DESC(?techCount)
    LIMIT 10""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_BPIntervention_RiskSubgroup.append(sTG)
        sRG.has_BPIntervention_RiskSubgroup = list(set(sRG.has_BPIntervention_RiskSubgroup))


## [x] Mitigations --> Techs

In [42]:
with onto:
    class has_BPIntervention_TechGroup(onto.BP_Intervention >> onto.TechGroup):
        label = ["Technology groups that can be used by this intervention"]
        pass
    class has_BPIntervention_TechSubgroup(onto.BP_Intervention >> onto.TechSubgroup):
        label = ["Technologysub groups that can be used by this intervention"]
        pass


In [43]:
for RG in [x.name for x in onto.BP_Intervention.instances()]:
    # @TODO bcp trop long itoo
    print(RG)
    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT ?interv ?techGroup (COUNT(?risk) AS ?techCount)
    WHERE {

    ?riskGroup a pbn:RiskGroup  .
    ?interv a pbn:BP_Intervention  .
    ?risk a pbn:Risk .
               
    ?technology pbn:has_TechGroup ?techGroup . 
    ?risk pbn:has_RiskTechnology ?technology .
    ?interv pbn:has_InterventionMitig ?mitig  .    
    ?risk pbn:has_RiskMitigation ?mitig .                    
            
    FILTER (?interv = pbn:"""+RG+""") 
    }
            
    GROUP BY ?techGroup
    ORDER BY DESC(?techCount)
    LIMIT 3""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_BPIntervention_TechGroup.append(sTG)
        sRG.has_BPIntervention_TechGroup = list(set(sRG.has_BPIntervention_TechGroup))
    print(A)

    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT ?interv ?techSubgroup (COUNT(?risk) AS ?techCount)
    WHERE {

    ?techSubgroup a pbn:TechSubgroup  .
    ?interv a pbn:BP_Intervention  .
    ?risk a pbn:Risk .

    ?technology pbn:has_TechSubgroup ?techSubgroup . 
    ?risk pbn:has_RiskTechnology ?technology .                         
    ?risk pbn:has_RiskMitigation ?mitig .
    ?interv pbn:has_InterventionMitig ?mitig  .                        
            
    FILTER (?interv = pbn:"""+RG+""") 
    }
            
    GROUP BY ?techSubgroup
    ORDER BY DESC(?techCount)
    LIMIT 10""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_BPIntervention_TechSubgroup.append(sTG)
        sRG.has_BPIntervention_TechSubgroup = list(set(sRG.has_BPIntervention_TechSubgroup))
    print(A)

PBN__BP_Intervention_21
[]
[]
PBN__BP_Intervention_30
[[WIP_Step1.PBN__BP_Intervention_30, WIP_Step1.PBN__TechGroup_0, 9909], [WIP_Step1.PBN__BP_Intervention_30, WIP_Step1.PBN__Technology_1447, 3375], [WIP_Step1.PBN__BP_Intervention_30, WIP_Step1.PBN__TechGroup_12, 3105]]
[[WIP_Step1.PBN__BP_Intervention_30, WIP_Step1.PBN__TechSubgroup_25, 312], [WIP_Step1.PBN__BP_Intervention_30, WIP_Step1.PBN__TechSubgroup_10, 246], [WIP_Step1.PBN__BP_Intervention_30, WIP_Step1.PBN__TechSubgroup_1, 216], [WIP_Step1.PBN__BP_Intervention_30, WIP_Step1.PBN__TechSubgroup_30, 135], [WIP_Step1.PBN__BP_Intervention_30, WIP_Step1.PBN__TechSubgroup_0, 122], [WIP_Step1.PBN__BP_Intervention_30, WIP_Step1.PBN__TechSubgroup_75, 96], [WIP_Step1.PBN__BP_Intervention_30, WIP_Step1.PBN__TechSubgroup_2, 92], [WIP_Step1.PBN__BP_Intervention_30, WIP_Step1.PBN__TechSubgroup_18, 86], [WIP_Step1.PBN__BP_Intervention_30, WIP_Step1.PBN__TechSubgroup_90, 54], [WIP_Step1.PBN__BP_Intervention_30, WIP_Step1.PBN__TechSubgroup_43,

## [x] Mitigations --> Stakeholders

In [44]:
with onto:
    class has_BPIntervention_StakeholderGroup(onto.BP_Intervention >> onto.StakeholderGroup):
        label = ["People groups that can be supported by this intervention"]
        pass
    class has_BPIntervention_StakeholderSubgroup(onto.BP_Intervention >> onto.StakeholderSubgroup):
        label = ["People subgroups that can be supported by this intervention"]
        pass


In [45]:
for RG in [x.name for x in onto.BP_Intervention.instances()]:

    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

        SELECT DISTINCT ?interv ?shGroup   (COUNT(?risk) AS ?mCount)
        WHERE {

        ?shGroup a pbn:StakeholderGroup  .
        ?risk a pbn:Risk  .
        ?mitig a pbn:Mitigation  .
        ?interv a pbn:BP_Intervention  .
                
        ?interv pbn:has_InterventionMitig ?mitig  .
        ?risk pbn:has_RiskMitigation ?mitig .
        ?risk pbn:has_RiskOwner ?sh .
        ?sh  pbn:has_StakeholderGroup ?shGroup .   
               
        FILTER (?interv = pbn:"""+RG+""") 
        }
        
        GROUP BY ?shGroup
        ORDER BY DESC(?mCount)           

        LIMIT 5""")

    for a in A:
        [sRG, sTG, c] = a
        sRG.has_BPIntervention_StakeholderGroup.append(sTG)
        sRG.has_BPIntervention_StakeholderGroup = list(set(sRG.has_BPIntervention_StakeholderGroup))


In [46]:
A

[[WIP_Step1.PBN__BP_Intervention_45, WIP_Step1.PBN__StakeholderGroup_12, 32],
 [WIP_Step1.PBN__BP_Intervention_45, WIP_Step1.PBN__StakeholderGroup_6, 26],
 [WIP_Step1.PBN__BP_Intervention_45, WIP_Step1.PBN__StakeholderGroup_1, 23],
 [WIP_Step1.PBN__BP_Intervention_45, WIP_Step1.PBN__StakeholderGroup_2, 19],
 [WIP_Step1.PBN__BP_Intervention_45, WIP_Step1.PBN__StakeholderGroup_13, 14]]

In [47]:
for RG in [x.name for x in onto.BP_Intervention.instances()]:

    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

        SELECT DISTINCT ?interv ?shSubgroup   (COUNT(?risk) AS ?mCount)
        WHERE {

        ?shSubgroup a pbn:StakeholderSubgroup  .
        ?risk a pbn:Risk  .
        ?mitig a pbn:Mitigation  .
        ?interv a pbn:BP_Intervention  .
               
        ?sh  pbn:has_StakeholderSubgroup ?shSubgroup .        
        ?risk pbn:has_RiskOwner ?sh .
        ?risk pbn:has_RiskMitigation ?mitig .
        ?interv pbn:has_InterventionMitig ?mitig  .

        FILTER (?interv = pbn:"""+RG+""") 
        }
        
        GROUP BY ?shSubgroup
        ORDER BY DESC(?mCount)           

        LIMIT 5""")

    for a in A:
        [sRG, sTG, c] = a
        sRG.has_BPIntervention_StakeholderSubgroup.append(sTG)
        sRG.has_BPIntervention_StakeholderSubgroup = list(set(sRG.has_BPIntervention_StakeholderSubgroup))


In [48]:
A

[[WIP_Step1.PBN__BP_Intervention_45,
  WIP_Step1.PBN__StakeholderSubgroup_21,
  17],
 [WIP_Step1.PBN__BP_Intervention_45,
  WIP_Step1.PBN__StakeholderSubgroup_87,
  13],
 [WIP_Step1.PBN__BP_Intervention_45, WIP_Step1.PBN__StakeholderSubgroup_2, 12],
 [WIP_Step1.PBN__BP_Intervention_45,
  WIP_Step1.PBN__StakeholderSubgroup_14,
  10],
 [WIP_Step1.PBN__BP_Intervention_45, WIP_Step1.PBN__StakeholderSubgroup_47, 8]]

In [49]:
for k in onto.data_properties():
    print(str(k))

WIP_Step1.has_BenefDetails
WIP_Step1.has_RiskDescription
WIP_Step1.has_RiskName
WIP_Step1.has_ArticleDOI
WIP_Step1.has_ArticleYear
WIP_Step1.has_ArticleTitle
WIP_Step1.has_ArticleLLMSummary
WIP_Step1.has_ArticleLLMShortSummary
WIP_Step1.has_MitigationPrinciple
WIP_Step1.has_BPTitle
WIP_Step1.has_BPDescription
WIP_Step1.has_BPContext
WIP_Step1.has_BPReason
WIP_Step1.has_BPProblem
WIP_Step1.has_BPValueProposition
WIP_Step1.has_BPCapabilities
WIP_Step1.has_BPRestriction
WIP_Step1.has_BPDemonstrated
WIP_Step1.has_BP_EnablerTitle
WIP_Step1.has_BP_InterventionTitle
WIP_Step1.has_BP_InterventionKey
WIP_Step1.has_BP_ThemeTitle
WIP_Step1.has_BP_CategoryTitle
WIP_Step1.has_PBNCategoryTitle
WIP_Step1.has_BlueprintSummary
WIP_Step1.has_BlueprintDetails
WIP_Step1.has_bibRef
WIP_Step1.has_for_synonym


In [50]:
print("Done")

Done


# Articles

### [x] Articles -> Risks

In [51]:
with onto:
    class has_Article_RiskGroup(onto.Article >> onto.RiskGroup):
        label = ["Risk groups covered by this article"]
        pass
    class has_Article_RiskSubgroup(onto.Article >> onto.RiskSubgroup):
        label = ["Risks subgroups covered by this article"]
        pass


In [52]:
for RG in [x.name for x in onto.Article.instances()]:

    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

        SELECT DISTINCT ?article ?rGroup   (COUNT(?risk) AS ?mCount)
        WHERE {

        ?rGroup a pbn:RiskGroup  .
        ?risk a pbn:Risk .
        ?article a pbn:Article  .
                
        ?risk pbn:has_RiskSource ?article .
        ?risk  pbn:has_RiskGroup ?rGroup .   
               
        FILTER (?article = pbn:"""+RG+""") 
        }
        
        GROUP BY ?rGroup
        ORDER BY DESC(?mCount)           

        LIMIT 3""")

    for a in A:
        [sRG, sTG, c] = a
        sRG.has_Article_RiskGroup.append(sTG)
        sRG.has_Article_RiskGroup = list(set(sRG.has_Article_RiskGroup))
A

[]

In [53]:
P = onto.has_RiskInArticle
P.get_inverse_property()

WIP_Step1.has_RiskSource

In [54]:
for RG in [x.name for x in onto.Article.instances()]:

    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

        SELECT DISTINCT ?article ?rGroup   (COUNT(?risk) AS ?mCount)
        WHERE {

        ?rGroup a pbn:RiskSubgroup  .
        ?risk a pbn:Risk .
        ?article a pbn:Article  .
                
        ?risk pbn:has_RiskSource ?article .
        ?risk  pbn:has_RiskSubgroup ?rGroup .   
               
        FILTER (?article = pbn:"""+RG+""") 
        }
        
        GROUP BY ?rGroup
        ORDER BY DESC(?mCount)           

        LIMIT 3""")

    for a in A:
        [sRG, sTG, c] = a
        sRG.has_Article_RiskSubgroup.append(sTG)
        sRG.has_Article_RiskSubgroup = list(set(sRG.has_Article_RiskSubgroup))
A

[]

### [x] Articles -> Stakeholders

In [55]:
with onto:
    class has_Article_StakeholderGroup(onto.Article >> onto.StakeholderGroup):
        label = ["People groups that can be supported by this article"]
        pass
    class has_Article_StakeholderSubgroup(onto.Article >> onto.StakeholderSubgroup):
        label = ["People subgroups that can be supported by this article"]
        pass


In [56]:
for RG in [x.name for x in onto.Article.instances()]:

    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

        SELECT DISTINCT ?article ?shGroup   (COUNT(?risk) AS ?mCount)
        WHERE {

        ?shGroup a pbn:StakeholderGroup  .
        ?sh a pbn:Stakeholder  .
        ?risk a pbn:Risk .
        ?article a pbn:Article  .
                
        ?risk pbn:has_RiskSource ?article .
        ?risk pbn:has_RiskSubject ?sh .
        ?sh  pbn:has_StakeholderGroup ?shGroup .   
               
        FILTER (?article = pbn:"""+RG+""") 
        }
        
        GROUP BY ?shGroup
        ORDER BY DESC(?mCount)           

        LIMIT 3""")

    for a in A:
        [sRG, sTG, c] = a
        sRG.has_Article_StakeholderGroup.append(sTG)
        sRG.has_Article_StakeholderGroup = list(set(sRG.has_Article_StakeholderGroup))
A

[]

In [57]:
for RG in [x.name for x in onto.Article.instances()]:

    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

        SELECT DISTINCT ?article ?shSGroup   (COUNT(?sh) AS ?mCount)
        WHERE {

        ?shSGroup a pbn:StakeholderSubgroup  .
        ?sh a pbn:Stakeholder  .
        ?rsk a pbn:Risk .
        ?article a pbn:Article  .
                
        ?risk pbn:has_RiskSource ?article .
        ?risk pbn:has_RiskSubject ?sh .
        ?sh  pbn:has_StakeholderSubgroup ?shSGroup .   
               
        FILTER (?article = pbn:"""+RG+""") 
        }
        
        GROUP BY ?shSGroup
        ORDER BY DESC(?mCount)           

        LIMIT 5""")

    for a in A:
        [sRG, sTG, c] = a
        sRG.has_Article_StakeholderGroup.append(sTG)
        sRG.has_Article_StakeholderGroup = list(set(sRG.has_Article_StakeholderGroup))
A

[]

### [x] Articles -> CAO

In [58]:
with onto:
    class has_Article_CAO_Group(onto.Article >> onto.CAO_Group):
        label = ["City anatomy layer that can be covered in this article"]
        pass
    class has_Article_CAO_Subgroup(onto.Article >> onto.CAO_Subgroup):
        label = ["City anatomy sublayer that can be covered in this article"]
        pass




In [59]:
for RG in [x.name for x in onto.Article.instances()]:

    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

        SELECT DISTINCT ?article ?CAOGroup   (COUNT(?risk) AS ?mCount)
        WHERE {

        ?CAOGroup a pbn:CAO_Group  .
        ?risk a pbn:Risk .
        ?article a pbn:Article  .
                
        ?risk pbn:has_RiskSource ?article .
        ?risk  pbn:has_RiskCAOGroup ?CAOGroup .   
               
        FILTER (?article = pbn:"""+RG+""") 
        }
        
        GROUP BY ?CAOGroup
        ORDER BY DESC(?mCount)           

        LIMIT 3""")

    for a in A:
        [sRG, sTG, c] = a
        sRG.has_Article_CAO_Group.append(sTG)
        sRG.has_Article_CAO_Group = list(set(sRG.has_Article_CAO_Group))
A

[]

In [60]:
for RG in [x.name for x in onto.Article.instances()]:

    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

        SELECT DISTINCT ?article ?caoSGroup   (COUNT(?risk) AS ?mCount)
        WHERE {

        ?caoSGroup a pbn:CAO_Subgroup  .
        ?risk a pbn:Risk .
        ?article a pbn:Article  .
                
        ?risk pbn:has_RiskSource ?article .
        ?risk  pbn:has_RiskCAOSubgroup ?caoSGroup .   
               
        FILTER (?article = pbn:"""+RG+""") 
        }
        
        GROUP BY ?caoSGroup
        ORDER BY DESC(?mCount)           

        LIMIT 3""")

    for a in A:
        [sRG, sTG, c] = a
        sRG.has_Article_CAO_Subgroup.append(sTG)
        sRG.has_Article_CAO_Subgroup = list(set(sRG.has_Article_CAO_Subgroup))
A

[]

### [x] Articles -> Techs

In [61]:
with onto:
    class has_Article_TechGroup(onto.Article >> onto.TechGroup):
        label = ["Tech groups that can be supported by this article"]
        pass
    class has_Article_TechSubgroup(onto.Article >> onto.TechSubgroup):
        label = ["Tech subgroups that can be supported by this article"]
        pass


In [62]:
for RG in [x.name for x in onto.Article.instances()]:

    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

        SELECT DISTINCT ?article ?techGroup   (COUNT(?tech) AS ?mCount)
        WHERE {

        ?techGroup a pbn:TechGroup  .
        ?risk a pbn:Risk .
        ?tech a pbn:Technology .
        ?article a pbn:Article  .
                
        ?risk pbn:has_RiskSource ?article . 
        ?risk pbn:has_RiskTechnology ?tech .
        ?tech pbn:has_TechGroup ?techGroup .

        FILTER (?article = pbn:"""+RG+""") 
        }
        
        GROUP BY ?techGroup
        ORDER BY DESC(?mCount)           

        LIMIT 3""")

    for a in A:
        [sRG, sTG, c] = a
        sRG.has_Article_TechGroup.append(sTG)
        sRG.has_Article_TechGroup = list(set(sRG.has_Article_TechGroup))
A

[]

In [63]:
for RG in [x.name for x in onto.Article.instances()]:

    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

        SELECT DISTINCT ?article ?techGroup   (COUNT(?risk) AS ?mCount)
        WHERE {

        ?techGroup a pbn:TechSubgroup  .
        ?risk a pbn:Risk .
        ?tech a pbn:Technology .
        ?article a pbn:Article  .
                
        ?risk pbn:has_RiskSource ?article . 
        ?risk pbn:has_RiskTechnology ?tech .
        ?tech pbn:has_TechSubgroup ?techGroup .

        FILTER (?article = pbn:"""+RG+""") 
        }
        
        GROUP BY ?techGroup
        ORDER BY DESC(?mCount)           

        LIMIT 5""")

    for a in A:
        [sRG, sTG, c] = a
        sRG.has_Article_TechSubgroup.append(sTG)
        sRG.has_Article_TechSubgroup = list(set(sRG.has_Article_TechSubgroup))
A

[]

### [x] Articles -> Interventions

In [64]:
with onto:
    class has_Article_BPIntervention(onto.Article >> onto.BP_Intervention):
        label = ["Interventions that can be covered in this article"]
        pass



In [65]:
for RG in [x.name for x in onto.Article.instances()]:
    #@TODO another long query
    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

        SELECT DISTINCT ?article ?interv   (COUNT(?risk) AS ?mCount)
        WHERE {
        
        ?article a pbn:Article  .        
        ?risk a pbn:Risk .
        ?tech a pbn:Technology .

        ?risk pbn:has_RiskSource ?article . 
        ?risk pbn:has_RiskMitigation ?mitig .
        ?interv pbn:has_InterventionMitig ?mitig  .

        FILTER (?article = pbn:"""+RG+""") 
        }
        
        GROUP BY ?interv
        ORDER BY DESC(?mCount)           

        LIMIT 5""")

    for a in A:
        [sRG, sTG, c] = a
        sRG.has_Article_BPIntervention.append(sTG)
        sRG.has_Article_BPIntervention = list(set(sRG.has_Article_BPIntervention))
A

[]

# Closing

In [66]:
u.cOnto(onto)


# Overview

*  _PBNThing_  --  87838 instances.
*  _BenefReturn_  --  1606 instances.
*  _Benef_  --  1409 instances.
*  _Article_  --  377 instances.
*  _Risk_  --  21145 instances.
*  _ISO_Scale_  --  5 instances.
*  _RiskHealth_  --  14 instances.
*  _RiskType_  --  22 instances.
*  _Stakeholder_  --  16125 instances.
*  _Stakeholder_Type_  --  3 instances.
*  _Technology_  --  23140 instances.
*  _ISO_Impact_  --  24 instances.
*  _ISO_Purpose_  --  18 instances.
*  _StakeholderGroup_  --  18 instances.
*  _StakeholderSubgroup_  --  168 instances.
*  _TechGroup_  --  15 instances.
*  _TechSubgroup_  --  127 instances.
*  _Mitigation_  --  22950 instances.
*  _BP_Enabler_  --  33 instances.
*  _BP_Transmission_  --  12 instances.
*  _Blueprint_  --  74 instances.
*  _BP_Scale_  --  3 instances.
*  _BP_Phase_  --  4 instances.
*  _BP_Permanent_  --  4 instances.
*  _BP_Type_  --  2 instances.
*  _BP_Intervention_  --  46 instances.
*  _PBNCategory_  --  330 instances.
*  _RiskGroup_

In [72]:
onto.save("./WIP_w_SPARQL.owl")

# Debug


In [69]:
1

1

In [71]:
for C in list(onto.Article.instances())[:5]:
    print([x for x in C.has_Article_BPIntervention])
D = list(C.has_Article_BPIntervention)[0].is_a
D

[WIP_Step1.PBN__BP_Intervention_29, WIP_Step1.PBN__BP_Intervention_31, WIP_Step1.PBN__BP_Intervention_23, WIP_Step1.PBN__BP_Intervention_1, WIP_Step1.PBN__BP_Intervention_9]
[WIP_Step1.PBN__BP_Intervention_29, WIP_Step1.PBN__BP_Intervention_9, WIP_Step1.PBN__BP_Intervention_1, WIP_Step1.PBN__BP_Intervention_22, WIP_Step1.PBN__BP_Intervention_38]
[WIP_Step1.PBN__BP_Intervention_29, WIP_Step1.PBN__BP_Intervention_31, WIP_Step1.PBN__BP_Intervention_11, WIP_Step1.PBN__BP_Intervention_30, WIP_Step1.PBN__BP_Intervention_38]
[WIP_Step1.PBN__BP_Intervention_29, WIP_Step1.PBN__BP_Intervention_31, WIP_Step1.PBN__BP_Intervention_35, WIP_Step1.PBN__BP_Intervention_1, WIP_Step1.PBN__BP_Intervention_9]
[WIP_Step1.PBN__BP_Intervention_29, WIP_Step1.PBN__BP_Intervention_37, WIP_Step1.PBN__BP_Intervention_30, WIP_Step1.PBN__BP_Intervention_20, WIP_Step1.PBN__BP_Intervention_38]


[WIP_Step1.BP_Intervention]